In [1]:
import os

from dotenv import load_dotenv
from etg_client import GuestRoom

load_dotenv()

# ETG API Credentials
ETG_KEY_ID = os.environ["ETG_KEY_ID"]
ETG_API_KEY = os.environ["ETG_API_KEY"]
ETG_REQUEST_TIMEOUT = 30.0

# Gemini API Key
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]
GEMINI_REQUEST_TIMEOUT = 30.0

# Search Parameters
CITY = "Шанхай"
REGION_ID: int | None = None

CHECKIN_DATE = "2026-03-15"
CHECKOUT_DATE = "2026-03-17"

CURRENCY = "EUR"
LANGUAGE = "ru"
RESIDENCY = "RU"

GUESTS: list[GuestRoom] = [{"adults": 2, "children": []}]
LIMIT = 1000

# User preferences for AI
USER_PREFERENCES = "Большая кровать, не было плесени. Не больше 10 мин пешком до метро. Отель находился рядом с центром"

# Reviews settings
REVIEWS_PER_SEGMENT = 30
REVIEWS_MAX_AGE_YEARS = 5
NEUTRAL_RATING_THRESHOLD = 7.0
NEGATIVE_RATING_THRESHOLD = 5.0

# Filters
MIN_PRICE: float | None = 140.0  # None = no minimum
MAX_PRICE: float | None = 300.0  # None = no maximum

In [2]:
from etg_client import (
    ETGClient,
    Hotel,
    HotelContent,
)

client = ETGClient(ETG_KEY_ID, ETG_API_KEY, timeout=ETG_REQUEST_TIMEOUT)

In [3]:
def find_region_id(client: ETGClient, city_name: str, language: str) -> int | None:
    """Find region ID for a city name."""
    print(f"Looking up region ID for '{city_name}'...")
    regions = client.suggest_region(city_name, language)

    if not regions:
        print(f"  No regions found for '{city_name}'")
        return None

    # Only accept City type
    for region in regions:
        if region["type"] == "City":
            region_id = region["id"]
            print(f"  Found: {region['name']} ({region.get('country_code', '')}), region_id={region_id}")
            return region_id

    # No city found - show available options
    print(f"  No city found. Available regions:")
    for r in regions[:5]:
        print(f"    - {r['name']} (type: {r['type']}, id: {r['id']})")

    return None

In [4]:
# Find region
region_id = REGION_ID or find_region_id(client, CITY, LANGUAGE)
if not region_id:
    raise ValueError(f"Could not find region for '{CITY}'")

print(f"\nSearching hotels in {CITY}...")
print(f"  Dates: {CHECKIN_DATE} to {CHECKOUT_DATE}")
print(f"  Currency: {CURRENCY}, Limit: {LIMIT}")

Looking up region ID for 'Шанхай'...
  [ETG] /api/b2b/v3/search/multicomplete/ - 200 in 0.86s
  Found: Шанхай (CN), region_id=3145

Searching hotels in Шанхай...
  Dates: 2026-03-15 to 2026-03-17
  Currency: EUR, Limit: 1000


In [5]:
import pandas as pd

# Search hotels
results = client.search_hotels_by_region(
    region_id=region_id,
    checkin=CHECKIN_DATE,
    checkout=CHECKOUT_DATE,
    residency=RESIDENCY,
    guests=GUESTS,
    currency=CURRENCY,
    language=LANGUAGE,
    hotels_limit=LIMIT,
)

hotels: list[Hotel] = results.get("hotels", [])
total_hotels = results.get("total_hotels", len(hotels))

if not hotels:
    print("No hotels found for the given criteria.")
    df_hotels = pd.DataFrame()
else:
    print(f"Found {len(hotels)} hotels (total available: {total_hotels})\n")

    # Create DataFrame
    hotels_data = []
    for hotel in hotels:
        rates = hotel.get("rates", [])
        if rates:
            first_rate = rates[0]
            payment_types = first_rate.get("payment_options", {}).get("payment_types", [])
            price = float(payment_types[0].get("show_amount", 0)) if payment_types else None
            room_name = first_rate.get("room_name", "")
            meal = first_rate.get("meal", "")
        else:
            price = None
            room_name = ""
            meal = ""
        
        hotels_data.append({
            "hotel_id": hotel["id"],
            "hid": hotel["hid"],
            "price": price,
            "currency": CURRENCY,
            "room": room_name[:40] if room_name else "",
            "meal": meal,
        })

    df_hotels = pd.DataFrame(hotels_data)
    df_hotels = df_hotels.sort_values("price", ascending=True).reset_index(drop=True)
    df_hotels.index += 1  # Start from 1

    # Display settings
    pd.set_option("display.max_colwidth", 50)
    pd.set_option("display.max_rows", 100)

df_hotels

  [ETG] /api/b2b/v3/search/serp/region/ - 200 in 6.03s
Found 508 hotels (total available: 508)



,hotel_id,hid,price,currency,room,meal
1,yibaihotel,13410286,26.0,EUR,Одноместный номер Economy (односпальная,nomeal
2,ybjdshhqjcgzzxd,13410287,26.0,EUR,Двухместный номер Economy (двуспальная к,nomeal
3,xilaige_hostel,8790358,32.0,EUR,Двухместный номер Standard (двуспальная,nomeal
4,estate_hotelshanghai_chuansha_store,13266650,35.0,EUR,Двухместный номер Standard (двуспальная,nomeal
5,lanyi_inn_hotel_shanghai_zhangze,8791985,38.0,EUR,Двухместный номер Standard (2 отдельные,nomeal
...,...,...,...,...,...,...
504,j_hotel_shanghai_tower,10488483,1130.0,EUR,Двухместный номер State (двуспальная кро,breakfast
505,the_shanghai_edition,9011716,1273.0,EUR,Двухместный номер с видом на башню Перл,nomeal
506,junshe_holiday_villa,13267456,2233.0,EUR,Люкс с 4 комнатами,breakfast
507,shanghai_youjian_linjian_homestay,13278451,3872.0,EUR,Вилла,nomeal


In [6]:
from hotels import filter_hotels_by_price

# Apply price filter
hotels = filter_hotels_by_price(hotels, MIN_PRICE, MAX_PRICE)
print(f"After price filter: {len(hotels)} hotels")

After price filter: 216 hotels


In [7]:
from hotels import fetch_hotel_content

In [8]:
hotel_hids = [h["hid"] for h in hotels]

content_map = fetch_hotel_content(client, hotel_hids, LANGUAGE)

  [ETG] /api/content/v1/hotel_content_by_ids/ - 200 in 1.58s
  [ETG] /api/content/v1/hotel_content_by_ids/ - 200 in 0.91s
  [ETG] /api/content/v1/hotel_content_by_ids/ - 200 in 0.45s


In [9]:
content_data = []
for hid, content in content_map.items():
    content_data.append({
        "hid": hid,
        "name": content.get("name", "")[:40],
        "stars": content.get("star_rating", 0),
        "kind": content.get("kind", ""),
        "address": content.get("address", "")[:50],
        "latitude": content.get("latitude"),
        "longitude": content.get("longitude"),
        "check_in": content.get("check_in_time", ""),
        "check_out": content.get("check_out_time", ""),
    })

df_content = pd.DataFrame(content_data)

# Merge with prices from df_hotels
df_full = df_hotels.merge(df_content, on="hid", how="left")
df_full = df_full[["hotel_id", "name", "stars", "kind", "price", "meal", "address"]]
df_full = df_full.sort_values("price", ascending=True).reset_index(drop=True)
df_full.index += 1

df_full

,hotel_id,name,stars,kind,price,meal,address
1,yibaihotel,NaN,NaN,NaN,26.0,nomeal,NaN
2,ybjdshhqjcgzzxd,NaN,NaN,NaN,26.0,nomeal,NaN
3,xilaige_hostel,NaN,NaN,NaN,32.0,nomeal,NaN
4,estate_hotelshanghai_chuansha_store,NaN,NaN,NaN,35.0,nomeal,NaN
5,lanyi_inn_hotel_shanghai_zhangze,NaN,NaN,NaN,38.0,nomeal,NaN
...,...,...,...,...,...,...,...
504,j_hotel_shanghai_tower,NaN,NaN,NaN,1130.0,breakfast,NaN
505,the_shanghai_edition,NaN,NaN,NaN,1273.0,nomeal,NaN
506,junshe_holiday_villa,NaN,NaN,NaN,2233.0,breakfast,NaN
507,shanghai_youjian_linjian_homestay,NaN,NaN,NaN,3872.0,nomeal,NaN


In [10]:
from reviews import fetch_reviews, filter_reviews, HotelReviewsFiltered

raw_reviews = fetch_reviews(client, hotel_hids, LANGUAGE)
reviews_map = filter_reviews(
    raw_reviews,
    max_age_years=REVIEWS_MAX_AGE_YEARS,
    reviews_per_segment=REVIEWS_PER_SEGMENT,
    neutral_threshold=NEUTRAL_RATING_THRESHOLD,
    negative_threshold=NEGATIVE_RATING_THRESHOLD,
)
print(f"Reviews: {sum(len(r['reviews']) for r in reviews_map.values())} filtered")

  [ETG] /api/content/v1/hotel_reviews_by_ids/ - 200 in 0.71s
  [ETG] /api/content/v1/hotel_reviews_by_ids/ - 200 in 0.33s
  [ETG] /api/content/v1/hotel_reviews_by_ids/ - 200 in 0.39s
  [ETG] /api/content/v1/hotel_reviews_by_ids/ - 200 in 0.35s
  [ETG] /api/content/v1/hotel_reviews_by_ids/ - 200 in 0.34s
  [ETG] /api/content/v1/hotel_reviews_by_ids/ - 200 in 0.34s
Reviews: 306 filtered


In [11]:
# Create DataFrame with reviews summary
reviews_data = []
for hid, data in reviews_map.items():
    hotel_id = next((h["id"] for h in hotels if h["hid"] == hid), "")
    reviews_data.append({
        "hotel_id": hotel_id,
        "hid": hid,
        "total": data["total_reviews"],
        "positive": data["positive_count"],
        "neutral": data["neutral_count"],
        "negative": data["negative_count"],
    })

df_reviews = pd.DataFrame(reviews_data)
df_reviews = df_reviews.sort_values("total", ascending=False).reset_index(drop=True)
df_reviews.index += 1

# Merge with hotel info
df_reviews_full = df_reviews.merge(
    df_content[["hid", "name", "stars"]],
    on="hid",
    how="left"
)
df_reviews_full = df_reviews_full[["hotel_id", "name", "stars", "total", "positive", "neutral", "negative"]]


def show_reviews(hotel_id: str, segment: str = "all", limit: int = 5) -> None:
    """
    Show reviews for a hotel.
    
    Args:
        hotel_id: Hotel ID (e.g. 'rosewood_hong_kong')
        segment: 'positive', 'negative', 'neutral', or 'all'
        limit: Number of reviews to show per segment
    """
    hid = next((h["hid"] for h in hotels if h["id"] == hotel_id), None)
    if not hid:
        print(f"Hotel '{hotel_id}' not found")
        return
    
    data = reviews_map.get(hid)
    if not data:
        print(f"No reviews for hotel '{hotel_id}'")
        return
    
    hotel_name = content_map.get(hid, {}).get("name", hotel_id)
    print(f"{'='*60}")
    print(f"{hotel_name}")
    print(f"Total: {data['total_reviews']} | +{data['positive_count']} / ~{data['neutral_count']} / -{data['negative_count']}")
    print(f"{'='*60}\n")
    
    reviews = data["reviews"]
    
    def print_segment(name: str, filter_fn, limit: int):
        segment_reviews = [r for r in reviews if filter_fn(r)][:limit]
        if not segment_reviews:
            return
        print(f"--- {name} ({len(segment_reviews)}) ---")
        for r in segment_reviews:
            rating = r["rating"]
            date = r["created"][:10]
            lang = r.get("_lang", "?")
            plus = r.get("review_plus", "").strip()
            minus = r.get("review_minus", "").strip()
            print(f"\n[{rating}/10] {date} [{lang}]")
            if plus:
                print(f"  + {plus[:300]}")
            if minus:
                print(f"  - {minus[:300]}")
        print()
    
    if segment in ("all", "positive"):
        print_segment("POSITIVE", lambda r: r["rating"] >= NEUTRAL_RATING_THRESHOLD, limit)
    if segment in ("all", "neutral"):
        print_segment("NEUTRAL", lambda r: NEGATIVE_RATING_THRESHOLD <= r["rating"] < NEUTRAL_RATING_THRESHOLD, limit)
    if segment in ("all", "negative"):
        print_segment("NEGATIVE", lambda r: r["rating"] < NEGATIVE_RATING_THRESHOLD, limit)


df_reviews_full

,hotel_id,name,stars,total,positive,neutral,negative
0,_0x6e9ec,Da Zhong Pudong Airport Hotel Shanghai,3,34,20,6,0
1,dorsett_shanghai,Dorsett Shanghai Hotel,4,27,26,0,0
2,central_hotel_shanghai,Central Hotel Shanghai,4,26,25,0,0
3,jin_jiang_tower_2,Jin Jiang Tower,4,19,19,0,0
4,the_eton_hotel,The Eton Hotel,5,19,15,1,0
...,...,...,...,...,...,...,...
193,le_meridien_sheshan_shanghai,Le Méridien Shanghai Sheshan,4,0,0,0,0
194,four_points_by_sheraton_shanghai_daning,"Four Points by Sheraton Shanghai, Daning",4,0,0,0,0
195,crowne_plaza_shanghai_fudan,Crowne Plaza Shanghai Fudan by IHG,4,0,0,0,0
196,radisson_blu_shanghai_pudong_jinqiao_2,Radisson Blu Shanghai Pudong Jinqiao,4,0,0,0,0


In [12]:
# Example: view reviews for a specific hotel
# show_reviews("four_seasons_st_petersburg")              # all segments, 5 per segment
# show_reviews("four_seasons_st_petersburg", "negative")  # only negative
# show_reviews("four_seasons_st_petersburg", "all", 10)   # all segments, 10 per segment

show_reviews("four_seasons_st_petersburg", limit=3)

Hotel 'four_seasons_st_petersburg' not found


In [13]:
class HotelCombined(Hotel, HotelContent):
    """Combined hotel data from search, content, and reviews."""
    reviews: HotelReviewsFiltered


def combine_hotel_data(
    hotels: list[Hotel],
    content_map: dict[int, HotelContent],
    reviews_map: dict[int, HotelReviewsFiltered],
) -> list[HotelCombined]:
    """Combine search results, content, and reviews."""
    print("Combining hotel data...")
    combined: list[HotelCombined] = []

    for hotel in hotels:
        hid = hotel["hid"]
        content = content_map.get(hid, {})
        reviews = reviews_map.get(hid, {
            "reviews": [],
            "total_reviews": 0,
            "positive_count": 0,
            "neutral_count": 0,
            "negative_count": 0,
        })

        combined.append({  # type: ignore[arg-type]
            **hotel,
            **content,
            "reviews": reviews,
        })

    print(f"  Combined {len(combined)} hotels")
    return combined

In [14]:
combined = combine_hotel_data(hotels, content_map, reviews_map)

Combining hotel data...
  Combined 216 hotels


In [15]:
import json

# Estimate token count for combined data
combined_json = json.dumps(combined, ensure_ascii=False)
char_count = len(combined_json)
# Rough estimate: ~4 chars per token for English, ~2-3 for Russian/mixed
estimated_tokens = char_count // 3

print(f"JSON size: {char_count:,} chars ({char_count / 1024 / 1024:.2f} MB)")
print(f"Estimated tokens: ~{estimated_tokens:,}")

JSON size: 18,207,871 chars (17.36 MB)
Estimated tokens: ~6,069,290


In [16]:
from hotels import presort_hotels
from scoring import score_hotels

In [19]:
# Pre-sort and limit to top 100 for LLM scoring
top_hotels = presort_hotels(combined, reviews_map, limit=100)
print(f"Pre-sorted: {len(top_hotels)} hotels for LLM scoring")

# Debug: show full prompts
DEBUG_SHOW_PROMPT = False

# Score hotels with progress
scoring_results = []
scoring_error = None

async for result in score_hotels(
    top_hotels,
    USER_PREFERENCES,
    currency=CURRENCY,
    min_price=MIN_PRICE,
    max_price=MAX_PRICE,
    batch_size=25,
):
    if result["type"] == "start":
        start = result["start"]
        print(f"\nScoring {start['total_hotels']} hotels in {start['total_batches']} batches (~{start['estimated_tokens']:,} tokens total)\n")
    elif result["type"] == "batch_start":
        bs = result["batch_start"]
        print(f"  → Batch {bs['batch']}/{bs['total_batches']}: {bs['hotels_in_batch']} hotels, ~{bs['estimated_tokens']:,} tokens")
        if DEBUG_SHOW_PROMPT:
            print(f"\n{'='*80}\nPROMPT (batch {bs['batch']}):\n{'='*80}\n{bs['prompt']}\n{'='*80}\n")
    elif result["type"] == "retry":
        retry = result["retry"]
        print(f"    ⚠ Retry {retry['attempt']}/{retry['max_attempts']}: {retry['error']}")
    elif result["type"] == "error":
        error = result["error"]
        scoring_error = error
        print(f"\n  ❌ ERROR [{error['error_type']}]: {error['message']}")
        break
    elif result["type"] == "progress":
        progress = result["progress"]
        print(f"  ✓ Batch {progress['batch']}/{progress['total_batches']} done ({progress['processed']}/{progress['total']} hotels)")
    elif result["type"] == "done":
        scoring_results = result["results"]

if scoring_error:
    print(f"\nScoring failed. Please check the error above.")
else:
    print(f"\nTotal scored: {len(scoring_results)} hotels")

Pre-sorted: 100 hotels for LLM scoring

Scoring 100 hotels in 4 batches (~115,977 tokens total)

  → Batch 1/4: 25 hotels, ~34,532 tokens
  ✓ Batch 1/4 done (25/100 hotels)
  → Batch 2/4: 25 hotels, ~27,924 tokens
  ✓ Batch 2/4 done (50/100 hotels)
  → Batch 3/4: 25 hotels, ~28,367 tokens
  ✓ Batch 3/4 done (75/100 hotels)
  → Batch 4/4: 25 hotels, ~25,154 tokens
  ✓ Batch 4/4 done (100/100 hotels)

Total scored: 96 hotels


In [20]:
from typing import Any
from hotels import get_ostrovok_url


def display_top_hotels(
    results: list[dict[str, Any]],
    hotels_data: list[dict[str, Any]],
    city: str,
    country_code: str = "DE",
    top_n: int = 10,
) -> pd.DataFrame:
    """Display top N scored hotels with details and Ostrovok links."""
    # Build hotel_id -> hid mapping
    hid_map = {h.get("id", ""): h.get("hid", 0) for h in hotels_data}
    name_map = {h.get("id", ""): h.get("name", h.get("id", "")) for h in hotels_data}
    
    print(f"\n{'='*80}")
    print(f"TOP {top_n} HOTELS")
    print(f"{'='*80}\n")

    data = []
    for i, hotel in enumerate(results[:top_n], 1):
        hotel_id = hotel.get("hotel_id", "")
        hid = hid_map.get(hotel_id, 0)
        score = hotel.get("score", 0)
        name = name_map.get(hotel_id, hotel_id)
        reasons = hotel.get("top_reasons", [])
        penalties = hotel.get("score_penalties", [])
        
        url = get_ostrovok_url(hotel_id, hid, city, country_code) if hid else ""

        # Print detailed info
        print(f"{i}. {name}")
        print(f"   Score: {score}/100")
        if reasons:
            print(f"   + {'; '.join(reasons[:3])}")
        if penalties:
            print(f"   - {'; '.join(penalties[:5])}")
        print(f"   {url}")
        print()
        
        # Collect for DataFrame
        data.append({
            "name": name[:40],
            "score": score,
            "reasons": "; ".join(reasons[:2])[:80] if reasons else "",
            "penalties": "; ".join(penalties[:5])[:120] if penalties else "",
            "url": url,
        })
    
    df = pd.DataFrame(data)
    df.index = range(1, len(df) + 1)
    total_found = len(hotels_data)
    selected = min(top_n, len(results))
    print(f"Всего найдено {total_found} отелей на эти даты. ")
    print(f"Подобраны лучшие {selected} по вашим критериям.")
    return df


# Get country code from first hotel's region
first_hotel = combined[0] if combined else {}
region = first_hotel.get("region", {})
country_code = region.get("country_code", "DE")

pd.set_option("display.max_colwidth", 100)
display_top_hotels(scoring_results, combined, CITY, country_code, top_n=10)


TOP 10 HOTELS

1. Central Hotel Shanghai
   Score: 98/100
   + Самый центр города, рядом с Нанкин-роуд; Большие кровати King Size в номерах Deluxe; Метро и достопримечательности в шаговой доступности
   https://ostrovok.ru/hotel/china/шанхай/mid7794335/central_hotel_shanghai/

2. Отель Shanghai Park
   Score: 95/100
   + Расположен в самом центре города; Наличие номеров с большими двуспальными кроватями; Рядом с Народной площадью и метро
   https://ostrovok.ru/hotel/china/шанхай/mid7489809/jin_jiang_park_hotel/

3. Апартотель Green Court Residence City Center Shanghai
   Score: 95/100
   + Отличное расположение в центре рядом с метро; Просторные номера с большими кроватями; Высокие оценки чистоты в отзывах
   https://ostrovok.ru/hotel/china/шанхай/mid7492944/green_court_serviced_apartment_2/

4. Radisson Collection Hyland Shanghai
   Score: 95/100
   + В самом центре на пешеходной улице; 5 минут пешком до метро; Есть номера с большой кроватью King Size
   - Некоторые номера меньше, че

,name,score,reasons,penalties,url
1,Central Hotel Shanghai,98,"Самый центр города, рядом с Нанкин-роуд; Большие кровати King Size в номерах Del",,https://ostrovok.ru/hotel/china/шанхай/mid7794335/central_hotel_shanghai/
2,Отель Shanghai Park,95,Расположен в самом центре города; Наличие номеров с большими двуспальными кроват,,https://ostrovok.ru/hotel/china/шанхай/mid7489809/jin_jiang_park_hotel/
3,Апартотель Green Court Residence City Ce,95,Отличное расположение в центре рядом с метро; Просторные номера с большими крова,,https://ostrovok.ru/hotel/china/шанхай/mid7492944/green_court_serviced_apartment_2/
4,Radisson Collection Hyland Shanghai,95,В самом центре на пешеходной улице; 5 минут пешком до метро,"Некоторые номера меньше, чем на фото",https://ostrovok.ru/hotel/china/шанхай/mid7584102/sofitel_shanghai_hyland/
5,New Asia,95,"2 км от центра, рядом с набережной Вайтань; 5-7 минут пешком до метро Tiantong R",Специфический запах в старом здании,https://ostrovok.ru/hotel/china/шанхай/mid8852136/golden_tulip_bund_new_asia/
6,Grand Central Hotel Shanghai,95,Расположен в самом центре у пешеходной улицы; Метро в непосредственной близости,Некоторые жалобы на сервис персонала,https://ostrovok.ru/hotel/china/шанхай/mid7812894/grand_central_hotel_shanghai/
7,Himalayas Hotel Shanghai,95,Метро Huamu в 100 метрах от отеля (1 минута пешком); Наличие номеров с большой к,,https://ostrovok.ru/hotel/china/шанхай/mid7807884/jumeirah_himalayas_hotel_shanghai/
8,Shanghai Fish Inn Bund,92,Пешая доступность до центра и набережной Бунд; Близко к метро и основным достопр,Некоторые гости жаловались на холод в номерах,https://ostrovok.ru/hotel/china/шанхай/mid7739085/shanghai_fish_inn_bund/
9,The Kunlun Jing An,92,100 метров до входа в метро Jing'an Temple; Находится в центре города,Здание 1988 года постройки,https://ostrovok.ru/hotel/china/шанхай/mid8876713/the_kunlun_hotel_jing_an/
10,Бутик-отель Ssaw Shanghai Bund,92,"2 км от центра, идеальное расположение для прогулок; Рядом с метро и садом Юй Юа",Жалобы на запах табака в номерах для некурящих,https://ostrovok.ru/hotel/china/шанхай/mid7755763/ssaw_hotel_shanghai/
